## Creating the Optimal Gov's Ball Scedule

In [3]:
#Bring in the neccessary libraries 
import pandas as pd
from io import StringIO
import urllib
from bs4 import BeautifulSoup 
import sqlite3 
import time
import numpy 
import signal
from urllib.request import Request, urlopen
from IPython.display import clear_output
import numpy as np
from scipy.stats import linregress


In [4]:
# Load package for progress bar
%run "progress_bar.py"
print ('Progress Bar Loaded')

Progress Bar Loaded


### Get Gov's Ball Artist Info

Could also look at Bonaroo and Lollapolloozas schedules

In [5]:
#Grab the artist info from the Gov's ball website
with urllib.request.urlopen("https://www.governorsballmusicfestival.com/lineup/interactive-lineup/") as url:
    s = url.read()
soup = BeautifulSoup(s, "lxml")


In [6]:
info = soup.findAll('div', {"class":"c-lineup__artist"})
#Put those names in a list, then a dataframe
govs_ball_data = pd.DataFrame([])
for i in info:
    artist = i.attrs['data-title']
    date = i.attrs["data-day-titles"]
    govs_ball_data = govs_ball_data.append(pd.DataFrame({'artist': artist, #Create the table
                                                         'date'  : date},
                                                  index=[0]),
                                     ignore_index=True)
govs_ball_data.head()

,artist,date
0,Tool,"[""Sunday, June 4th""]"
1,Chance The Rapper,"[""Friday, June 2nd""]"
2,Phoenix,"[""Saturday, June 3rd""]"
3,Childish Gambino,"[""Saturday, June 3rd""]"
4,Lorde,"[""Friday, June 2nd""]"


In [7]:
govs_ball_data["day"] = ((govs_ball_data["date"].str.slice(-5, -4)).astype(int) -1).astype(str)
govs_ball_data["date"] = govs_ball_data["date"].str.slice(2, -2)
govs_ball_data["festival"] = "Governor's Ball"
govs_ball_data.head()

,artist,date,day,festival
0,Tool,"Sunday, June 4th",3,Governor's Ball
1,Chance The Rapper,"Friday, June 2nd",1,Governor's Ball
2,Phoenix,"Saturday, June 3rd",2,Governor's Ball
3,Childish Gambino,"Saturday, June 3rd",2,Governor's Ball
4,Lorde,"Friday, June 2nd",1,Governor's Ball


### Get Schedule by Hour

In [ ]:
artist_time = pd.DataFrame([])

for i in log_progress(govs_ball_data.artist, every=1):
    artist = i.lower().replace(" ", "-").replace(u'ü', 'u')
    artist2 = i
    link = ('https://www.governorsballmusicfestival.com/artist/' + artist + "/") #create the link
    req = Request(link, headers={ 'User-Agent': 'Firefox/24.0' })
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, "lxml") #create the soup
    info_time = soup.findAll('p', {"class":"u-m0"})
    info_stage = soup.findAll('p', {"class":"u-mt0"})
    for div in info_time:
        text = div.text
        time = text.replace(" ", "").replace('\n', '')[7:]
    for div in info_stage:
        stage = div.text 
    artist_time = artist_time.append(pd.DataFrame({'time': time,
                                                   'artist' : artist2,
                                                   'stage' : stage}, #Create the table
                                     index=[0]))
        
govs_ball_data = govs_ball_data.merge(artist_time, on = "artist")

In [143]:
text

'Gov Ball NYC'

In [101]:
govs_ball_data.head()

,artist,date,day,festival,link,time
0,Tool,"Sunday, June 4th",3,Governor's Ball,9:15pm-11:00pm,9:15pm-11:00pm
1,Chance The Rapper,"Friday, June 2nd",1,Governor's Ball,9:45pm-11:00pm,9:45pm-11:00pm
2,Phoenix,"Saturday, June 3rd",2,Governor's Ball,9:15pm-11:00pm,9:15pm-11:00pm
3,Childish Gambino,"Saturday, June 3rd",2,Governor's Ball,9:15pm-11:00pm,9:15pm-11:00pm
4,Lorde,"Friday, June 2nd",1,Governor's Ball,7:30pm-8:45pm,7:30pm-8:45pm


### Repeat that process for Lolla and Bonnaroo

In [121]:
#Grab the artist info from the Lollapalooza website
with urllib.request.urlopen("https://www.lollapalooza.com/lineup/interactive-lineup/") as url:
    s = url.read()
soup = BeautifulSoup(s, "lxml")

info = soup.findAll('div', {"class":"c-lineup__artist"})
#Put those names in a list, then a dataframe
lolla_data = pd.DataFrame([])
for i in info:
    artist = i.attrs['data-title']
    date = i.attrs["data-day-titles"]
    lolla_data = lolla_data.append(pd.DataFrame({'artist': artist, #Create the table
                                                         'date'  : date},
                                                  index=[0]),
                                     ignore_index=True)
    
#Clean up the date information
lolla_data["day"] = np.where(lolla_data["date"].str.contains("Thursday"), "1",
                            np.where(lolla_data["date"].str.contains("Friday"), "2",
                                    np.where(lolla_data["date"].str.contains("Saturday"), "3", "4")))

lolla_data["date"] = lolla_data["date"].str.slice(2, -2)
lolla_data["date"] = lolla_data["date"].str.replace("\\", "")
lolla_data["date"] = lolla_data["date"].str.replace("8/6", "August 6th")
lolla_data["date"] = lolla_data["date"].str.replace("8/5", "August 5th")
lolla_data["date"] = lolla_data["date"].str.replace("8/4", "August 4th")
lolla_data["date"] = lolla_data["date"].str.replace("8/3", "August 3th")
lolla_data["festival"] = "Lollapalooza"
lolla_data.head()

,artist,date,day,festival
0,CHANCE THE RAPPER,"Saturday, August 5th",3,Lollapalooza
1,The Killers,"Friday, August 4th",2,Lollapalooza
2,MUSE,"Thursday, August 3th",1,Lollapalooza
3,Arcade Fire,"Sunday, August 6th",4,Lollapalooza
4,The xx,"Saturday, August 5th",3,Lollapalooza


In [133]:
artist_time = pd.DataFrame([])

for i in log_progress(lolla_data.artist, every=1):
    artist = i.lower().replace(" ", "-").replace(u'ü', 'u').replace(u'ī', 'i').replace(u'ī', 'i').replace("’", '').replace("é", 'e').replace("ä", 'a')
    if artist == "$uicideboy$":
        artist = i.lower().replace(" ", "-").replace("$", "s")
    if i in ["JoJo & The Pinecones", 'Lard Dog & The Band of Shy']:
        artist = i.lower().replace(" ", "-").replace("&", "and")
    artist2 = i
    link = ('https://www.lollapalooza.com/artist/' + artist + "/") #create the link
    req = Request(link, headers={ 'User-Agent': 'Firefox/24.0' })
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, "lxml") #create the soup
    info_time = soup.findAll('p', {"class":"u-m0"})
    info_stage = soup.findAll('p', {"class":"u-mt0"})
    for div in info_time:
        text = div.text
        time = text.replace(" ", "").replace('\n', '')[7:]
    for div in info_stage:
        stage = div.text 
    artist_time = artist_time.append(pd.DataFrame({'time': time,
                                                   'artist' : artist2,
                                                   'stage' : stage}, #Create the table
                                     index=[0]))
    
lolla_data = lolla_data.merge(artist_time, on = "artist")

In [134]:
#Grab the artist info from the  Bonnaroo website
with urllib.request.urlopen("https://www.bonnaroo.com/lineup/interactive/") as url:
    s = url.read()
soup = BeautifulSoup(s, "lxml")

info = soup.findAll('div', {"class":"c-lineup__artist"})
#Put those names in a list, then a dataframe
bonnaroo_data = pd.DataFrame([])
for i in info:
    artist = i.attrs['data-title']
    date = i.attrs["data-day-titles"]
    bonnaroo_data = bonnaroo_data.append(pd.DataFrame({'artist': artist, #Create the table
                                                         'date'  : date},
                                                  index=[0]),
                                     ignore_index=True)
    
bonnaroo_data["day"] = np.where(bonnaroo_data["date"].str.contains("Friday"), "1",
                                np.where(bonnaroo_data["date"].str.contains("Saturday"), "2", "3"))
bonnaroo_data["date"] = bonnaroo_data["date"].str.slice(2, -2)
bonnaroo_data["date"] = bonnaroo_data["date"].str.replace("\\", "")
bonnaroo_data["date"] = bonnaroo_data["date"].str.replace("6/9", "June 9th")
bonnaroo_data["date"] = bonnaroo_data["date"].str.replace("6/10", "June 10th")
bonnaroo_data["date"] = bonnaroo_data["date"].str.replace("6/11", "June 11th")
bonnaroo_data["festival"] = "Bonnaroo"
bonnaroo_data.head()

,artist,date,day,festival
0,U2,"Friday, June 9th",1,Bonnaroo
1,Red Hot Chili Peppers,"Saturday, June 10th",2,Bonnaroo
2,THE WEEKND,"Sunday, June 11th",3,Bonnaroo
3,Chance The Rapper,"Saturday, June 10th",2,Bonnaroo
4,Major Lazer,"Saturday, June 10th",2,Bonnaroo


In [139]:
artist_time = pd.DataFrame([])

for i in log_progress(bonnaroo_data.artist, every=1):
    artist = i.lower().replace(" ", "-").replace(u'ü', 'u').replace(u'ī', 'i').replace(u'ī', 'i').replace("’", '').replace("é", 'e').replace("ä", 'a')
    if artist == "$uicideboy$":
        artist = i.lower().replace(" ", "-").replace("$", "s")
    if i == "JoJo & The Pinecones":
        artist = i.lower().replace(" ", "-").replace("&", "and")
    if i == 'Shpongle (Simon Posford Presents The Shpongletron)':
        artist = "shpongle"
    if i == "Bluegrass Situation Roots Superjam Hosted By Ed Helms & Friends":
        artist = "bluegrass-situation-superjam-hosted-by-ed-helms-friends"
    artist2 = i
    link = ('https://www.bonnaroo.com/artist/' + artist + "/") #create the link
    req = Request(link, headers={ 'User-Agent': 'Firefox/24.0' })
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, "lxml") #create the soup
    info_time = soup.findAll('p', {"class":"u-m0"})
    info_stage = soup.findAll('p', {"class":"u-mt0"})
    for div in info_time:
        text = div.text
        time = text.replace(" ", "").replace('\n', '')[7:]
    for div in info_stage:
        stage = div.text 
    artist_time = artist_time.append(pd.DataFrame({'time': time,
                                                   'artist' : artist2,
                                                   'stage' : stage}, #Create the table
                                     index=[0]))
        
bonnaroo_data = bonnaroo_data.merge(artist_time, on = "artist")

HTTPError: HTTP Error 404: Not Found

In [138]:
artist2

'Bluegrass Situation Roots Superjam Hosted By Ed Helms & Friends'

#### Add Names to a SQLite Database

In [ ]:
#Create Connection
con = sqlite3.connect("pitchfork-data.db")
govs_ball_data.to_sql("govs_ball_data", con,if_exists='replace')
lolla_data.to_sql("lolla_data", con,if_exists='replace')
bonnaroo_data.to_sql("lolla_data", con,if_exists='replace')
all_festivals = govs_ball_data.append(lolla_data).append(bonnaroo_data)
all_festivals.to_sql("all_festivals", con,if_exists='replace')
con.close() #close db connection

## Pitchfork Crawler

The crawling happens in two distinct stages. In Stage 1, the code loops throught the reviews page on pitchfork.com to find links to all the reviews. Stage 2 goes to each link and pull various bits of information. There's lots more to pull, but this is a solid starting place. 

### Stage 1: 

In [ ]:
#Stage 1
con = sqlite3.connect("pitchfork-data.db") #connect to db

for i in log_progress(range(0,13), every=1): 
    #Use the range function to decide how many pages you want to go through
    #In this case, I'm going through the latest 100 reviews (12 per page)
    page_no = str(i)
    link = ('http://pitchfork.com/reviews/albums/?page=' + page_no) #create the link
    t0 = time.time()
    req = Request(link, headers={ 'User-Agent': 'Firefox/24.0' })
    webpage = urlopen(req).read()
    response_delay = time.time() - t0
    time.sleep(10*response_delay)  # wait 10x longer than it took them to respond
    soup = BeautifulSoup(webpage, "lxml") #create the soup
    info = soup.findAll('a', {"class":"album-link"}) #pull the album link
    for j in info:
            link = pd.DataFrame({'link': j.attrs['href']}, #Create the table
                         index=[0]).to_sql("link_table",
                                           con,
                                           if_exists = "append")
    clear_output() #clear ouput before rewriting progress
    print (i)

con.close() #close db connection

### Stage 2:

In [ ]:
BASE_URL = 'http://www.pitchfork.com'
con = sqlite3.connect("pitchfork-data.db")
links_table = pd.read_sql_query("SELECT DISTINCT * from link_table", con)
links = links_table["link"]
iterator = 0

class Timeout(Exception):  # handles timeout errors (e.g., server request is taking too long)
    pass

for i in log_progress(links[-156:], every=1):
    link = BASE_URL + i
    t0 = time.time()
    req = Request(link, headers={ 'User-Agent': 'Firefox/24.0' })
    webpage = urlopen(req).read()
    response_delay = time.time() - t0
    time.sleep(4*response_delay)  # wait Nx longer than it took them to respond
    soup = BeautifulSoup(webpage, "lxml") #same as above
    artist_info = soup.findAll('ul', {"class":"artist-links artist-list"}) #Artist Name
    album_info = soup.findAll('h1', {"class":"review-title"}) #Album Name
    score_info = soup.findAll('div', {"class":"score-circle"}) #Score
    pub_info = soup.findAll('span', {"class":"pub-date"}) # Publication Date
    genre_info = soup.findAll('ul', {"class":"genre-list before"}) #Genre
    for j in artist_info:
        artist = j.text
    for k in album_info:
        album = k.text
    for l in score_info:
        score = l.text
    for m in pub_info:
        pub_date = m.text
    for n in genre_info:
        genre = n.text
    clear_output()
    print (artist, ", ", album, response_delay, iterator)
    pd.DataFrame({'artist': artist, #Create the table
                  'album'  : album,
                  'score' : score,
                  'pub_date' : pub_date,
                  'genre' : genre,
                  'link' : link},
                 index=[0]).to_sql("album_table",
                                   con,
                                   if_exists = "append")
    iterator= iterator +1

con.close() 

# Data Management

In [53]:
con = sqlite3.connect("pitchfork-data.db")
#Pull the table we just wrote back it
album_table = pd.read_sql_query("SELECT * from album_table", con)
#Drop any duplicates that may have happened
album_table = album_table.drop_duplicates()
#Delete the index
del album_table["index"]
#reupload as album_table_clean
album_table.to_sql("album_table_clean", con, if_exists = "replace")

## Spotify API

In [40]:
import spotipy
import sys
import json
#Feed the Gov's Ball Artists into Spotify
spotify = spotipy.Spotify()#Create spotify object from spotipy 
spotify_data = pd.read_sql_query("SELECT DISTINCT artist from album_table_clean", con)

In [78]:
count = 0
for name in log_progress(artist_list["artist"][7342:], every = 1):
    try:
        search = spotify.search(q='artist:' + name, type='artist') #Search and grab the first result
        spot_name = pd.read_json(json.dumps(search["artists"]))["items"][0]["name"] #Name for reference
        popularity = pd.read_json(json.dumps(search["artists"]))["items"][0]["popularity"] #Popularity Metric
        genre = pd.read_json(json.dumps(search["artists"]))["items"][0]["genres"] #Genre list
        followers = pd.read_json(json.dumps(search["artists"]))["items"][0]["followers"]["total"] #Social Followers
        if len(genre) > 0:
            genre = "/".join(genre)
        else:
            genre = "Unknown"
        #Add that dat to the gov's ball table
        spotify_data.loc[spotify_data['artist'] == name, 'followers'] = followers
        spotify_data.loc[spotify_data['artist'] == name, 'genre'] = genre
        spotify_data.loc[spotify_data['artist'] == name, 'popularity'] = popularity
        count = count+1
    except Exception:
        pass

In [110]:
spotify_data = spotify_data[~spotify_data.followers.isnull()]
spotify_data["artist"] = spotify_data["artist"].str.upper()
spotify_data.head()

,artist,followers,genre,popularity
0,GRANDADDY,41891.0,alternative rock/anti-folk/britpop/chamber pop...,53.0
1,IMMOLATION,16597.0,avantgarde metal/black metal/black thrash/brut...,37.0
2,ELLIOTT SMITH,268151.0,alternative rock/anti-folk/chamber pop/folk-po...,62.0
3,WHY?,41171.0,abstract hip hop/anti-folk/bay area indie/cham...,49.0
4,ERASMO CARLOS,23114.0,bossa nova/brazilian indie/brega/forro/mpb/roc...,45.0


In [111]:
#reupload as artist_spotify_data
con = sqlite3.connect("pitchfork-data.db")
spotify_data.to_sql("artist_spotify_data", con, if_exists = "replace")

#### Cleaning Some Data

In [168]:
con = sqlite3.connect("pitchfork-data.db")
album_table = pd.read_sql_query("SELECT * from album_table_clean", con)
album_table["artist"] = album_table["artist"].str.upper()

In [169]:
artists = album_table["artist"].drop_duplicates()

In [170]:
fest_data = pd.read_sql_query("""SELECT UPPER(artist) AS "artist", day, date, festival from all_festivals""", con)
fest_data.head()


,artist,day,date,festival
0,TOOL,3,"Sunday, June 4th",Governor's Ball
1,CHANCE THE RAPPER,1,"Friday, June 2nd",Governor's Ball
2,PHOENIX,2,"Saturday, June 3rd",Governor's Ball
3,CHILDISH GAMBINO,2,"Saturday, June 3rd",Governor's Ball
4,LORDE,1,"Friday, June 2nd",Governor's Ball


In [171]:
fest_data["artist"] = np.where(fest_data["artist"] == "CHARLES BRADLEY & HIS EXTRAORDINAIRES",
                            "CHARLES BRADLEY",
                            fest_data["artist"])
fest_data["artist"] = np.where(fest_data["artist"] == "MARK RONSON VS KEVIN PARKER",
                            "MARK RONSON",
                            fest_data["artist"])
#x number of artists missing

In [172]:
print (fest_data[~(fest_data.artist.isin(artists))].shape[0], "artists without pitchfork data")

271 artists without pitchfork data


In [174]:
#reupload as govs_ball_enriched
#del fest_data["index"]
con = sqlite3.connect("pitchfork-data.db")
fest_data.to_sql("fest_data", con, if_exists = "replace")

### Merge in Pitchfork Reviews With Fest Data

In [175]:
query_pitchfork = """
        SELECT DISTINCT f.artist, f.festival,
               p.album, p.score, p.genre 
        FROM fest_data f 
        INNER JOIN album_table_clean p 
        ON (upper(p.artist) = f.artist)
        """

In [176]:
con = sqlite3.connect("pitchfork-data.db")
pitch_gov = pd.read_sql_query(query_pitchfork, con)
pitch_gov["score"] = pitch_gov["score"].astype(float)
pitch_gov.head(1)

,artist,festival,album,score,genre
0,STORMZY,Governor's Ball,Gang Signs & Prayer,7.6,Rap


In [177]:
score_metrics = pitch_gov.groupby(["artist",
                                   "genre",
                                   "festival"])["score"].agg(['mean',
                                                              "max",
                                                              "min",
                                                              'count' ]).reset_index()
score_metrics.head(1)

,artist,genre,festival,mean,max,min,count
0,A$AP FERG,Rap,Governor's Ball,7.066667,7.5,6.4,3


In [178]:
score_metrics.columns = [["artist",
                          "pf_genre",
                          "festival",
                          "pf_mean", 
                          "pf_max", 
                          "pf_min",
                          "pf_count"]]

score_metrics = round(score_metrics,1)
score_metrics.shape

(135, 7)

In [179]:
con = sqlite3.connect("pitchfork-data.db")
fd = pd.read_sql_query("SELECT * FROM artist_spotify_data", con)
fde = score_metrics.merge(fd, on = "artist", how = "left")
fde["genre"] = np.where(fde["pf_genre"].isnull(), fde["genre"], fde["pf_genre"])
del fde["pf_genre"]
#Clean up Genre Row
fde["genre"] = np.where(fde["genre"].isin(["Rap", "Pop/R&B", "Rock", "Electronic", "Metal"]),
                  fde["genre"],
                  np.where(fde["genre"].str.contains("hip hop"),
                     "Rap",
                     np.where(fde["genre"].str.contains("RockElectronic"),
                        "Rock",
                        np.where(fde["genre"].str.contains("rock"),
                           "Rock",
                           np.where(fde["genre"].str.contains("pop"),
                              "Pop/R&B",
                              np.where(fde["genre"].str.contains("electronic"),
                                 "Electronic",
                                 np.where(fde["genre"].str.contains("electronic"),
                                    "Electronic",
                                          np.where(fde["genre"].str.contains("comedy"),
                                                "Comedy",
                                    "Rock"))))))))

fde["genre"] = np.where(fde["genre"].isin(["Rock"]),
                        "Rock/Indie",
                        fde["genre"])
con.close()

In [180]:
fde["popularity_rank_fest"] = fde.groupby(["festival"])["popularity"].rank(ascending = False)
fde["fan_base_rank_fest"] = fde.groupby(["festival"])["followers"].rank(ascending = False)
fde["critical_metric"] = fde["pf_mean"]+fde["pf_count"] #Bump up artisits who have been reviewed several times
fde["critical_rank_fest"] = fde.groupby(["festival"])["critical_metric"].rank(ascending = False, )
fde["critical_rank_fest"] = np.where(fde["critical_rank_fest"].isnull(),
                                    fde["critical_rank_fest"].mean(),
                                    fde["critical_rank_fest"])

In [181]:
fde["total_ranking_score_fest"] = fde.iloc[:,-3:].sum(axis = 1)
fde["total_rank_fest"] = fde.groupby(["festival"])["total_ranking_score_fest"].rank(ascending = True)
fde.sort_values("total_rank_fest").head()

,artist,festival,pf_mean,pf_max,pf_min,pf_count,index,followers,genre,popularity,popularity_rank_fest,fan_base_rank_fest,critical_metric,critical_rank_fest,total_ranking_score_fest,total_rank_fest
132,WIZ KHALIFA,Lollapalooza,6.0,7.2,4.8,6,1226,4389509.0,Rap,87.0,3.0,1.0,12.0,9.0,22.0,1.0
116,THE WEEKND,Bonnaroo,7.6,8.5,6.2,7,292,5689483.0,Pop/R&B,93.0,1.0,1.0,14.6,2.0,17.6,1.0
131,WIZ KHALIFA,Governor's Ball,6.0,7.2,4.8,6,1226,4389509.0,Rap,87.0,1.0,1.0,12.0,6.0,19.0,1.0
117,THE XX,Bonnaroo,8.2,8.7,7.5,3,194,2418828.0,Rock/Indie,76.0,12.0,3.0,11.2,6.0,20.2,2.0
118,THE XX,Lollapalooza,8.2,8.7,7.5,3,194,2418828.0,Rock/Indie,76.0,14.5,3.0,11.2,14.0,28.2,2.0


In [182]:
con = sqlite3.connect("pitchfork-data.db")
#del fde["index"]
fde.to_sql("fest_data_enriched", con, if_exists = "replace")
con.close()

## Ad Hoc Questions

In [201]:
con = sqlite3.connect("pitchfork-data.db")

album_table = pd.read_sql_query("""SELECT upper(artist) as artist,
                                          count(album) as count, 
                                          avg(score) as score
                                    FROM album_table_clean
                                    GROUP BY artist""", con)
album_table["score"] = album_table["score"].astype(float)
album_table.head()

,artist,count,score
0,!!!,5,6.920000
1,+/-,3,6.366667
2,...AND YOU WILL KNOW US BY THE TRAIL OF DEAD,7,6.414286
3,"1,2,3",1,6.300000
4,10 IN THE SWEAR JAR,1,7.400000


In [224]:
spotify_data = pd.read_sql_query("SELECT * FROM artist_spotify_data", con)
corr = album_table.merge(spotify_data, on = "artist", how = "left")
corr = corr[~corr["followers"].isnull()]
corr = corr[corr["followers"] != 0]
corr.head()

,artist,count,score,index,followers,genre,popularity
1,+/-,3,6.366667,5699.0,1074520.0,classic funk rock/dance pop/funk/funk rock/qui...,74.0
2,...AND YOU WILL KNOW US BY THE TRAIL OF DEAD,7,6.414286,2318.0,23433.0,alternative rock/dance-punk/garage rock/indie ...,38.0
3,"1,2,3",1,6.300000,4447.0,3232.0,Unknown,23.0
4,10 IN THE SWEAR JAR,1,7.400000,7143.0,507.0,Unknown,9.0
5,11:11,1,4.900000,2505.0,120863.0,alternative metal/alternative rock/funk metal/...,54.0


In [231]:
print ("Corr. between log(followers) and popularity:",
       linregress(numpy.log(corr["followers"]),
                  corr["popularity"])[2])

print ("Corr. without log transformation:",
       linregress(corr["followers"],
                  corr["popularity"])[2])

Corr. between log(followers) and popularity: 0.913392910891
Corr. without log transformation: 0.440654051686


In [232]:
print ("Corr. between log(followers) and score:",
       linregress(numpy.log(corr["followers"]),
                  corr["score"])[2])

print ("Corr. without log transformation:",
       linregress(corr["followers"],
                  corr["score"])[2])

Corr. between log(followers) and popularity: -0.0215706802719
Corr. without log transformation: -0.0505064934547


In [233]:
print ("Corr. between popularity and score:",
       linregress(corr["popularity"],
                  corr["score"])[2])

Corr. between popularity and score: -0.0193207783818


In [237]:
print ("Corr. between count and score:",
       linregress(corr["count"],
                  corr["score"])[2])

Corr. between count and score: 0.0237439763623


In [238]:
corr["log_followers"] = numpy.log(corr["followers"])
con = sqlite3.connect("pitchfork-data.db")
#del fde["index"]
corr.to_sql("correlation_table", con, if_exists = "replace")
con.close()